In [15]:
from style_transfer import *
from IPython.display import Audio
from dataset import ItemClass, pad_wav
import torch
from model import *
import natsort
import glob
import librosa

In [2]:
config = {
    'n_mels': 80, 
    'n_fft': 1024,
    'fmin': 0,
    'fmax': 8000,
    'sr': 22050,
    'win_length': 1024,
    'hop_length': 256,
    'wav_max_length': 3024,
    'transcript_max_length': 580,
    'learning_rate': 1e-5, #1e-3, 
    'batch_size': 12,
    'weight_decay': 0, 
    'encoder_num_layers': 2,
    'encoder_hidden_dim': 256//2,
    'encoder_bidirectional': True,
    'encoder_dropout': 0.2,
    'decoder_hidden_dim': 512//2,  # must be 2 x encoder_hidden_dim
    'decoder_num_layers': 2,
    'decoder_multi_head': 1,
    'decoder_mlp_dim': 64,
    'asr_label_smooth': 0.1,
    'teacher_force_prob': 0.9,
    'ctc_weight': 0.5,
    'asr_weight': 0.7,
    'speaker_id_weight': 0.3
}

In [3]:
model_path = natsort.natsorted(glob.glob('asr/trained_models/wavglow-styletransfer/*'))[-1]
print(f"Loading model: {model_path}")
d = torch.load(model_path)

asr = LightningCTCLASMTL(**config)
asr.load_state_dict(d['state_dict'])

Loading model: asr/trained_models/wavglow-styletransfer/epoch=18-step=45049.ckpt


Reusing dataset librispeech_asr (/home/ani.vegesana.backup/.cache/huggingface/datasets/librispeech_asr/clean/2.1.0/1f4602f6b5fed8d3ab3e3382783173f2e12d9877e98775e34d7780881175096c)
Loading cached processed dataset at /home/ani.vegesana.backup/.cache/huggingface/datasets/librispeech_asr/clean/2.1.0/1f4602f6b5fed8d3ab3e3382783173f2e12d9877e98775e34d7780881175096c/cache-a795eaaac6783cf6.arrow
Loading cached processed dataset at /home/ani.vegesana.backup/.cache/huggingface/datasets/librispeech_asr/clean/2.1.0/1f4602f6b5fed8d3ab3e3382783173f2e12d9877e98775e34d7780881175096c/cache-edf4ca3d2f9596a0.arrow
Reusing dataset librispeech_asr (/home/ani.vegesana.backup/.cache/huggingface/datasets/librispeech_asr/clean/2.1.0/1f4602f6b5fed8d3ab3e3382783173f2e12d9877e98775e34d7780881175096c)
Loading cached processed dataset at /home/ani.vegesana.backup/.cache/huggingface/datasets/librispeech_asr/clean/2.1.0/1f4602f6b5fed8d3ab3e3382783173f2e12d9877e98775e34d7780881175096c/cache-230f3f90064d403f.arrow
Lo

<All keys matched successfully>

In [5]:
alice_hf_sample = asr.train_dataset.hf_ds[0]
alice_sample = asr.train_dataset[0]
alice_audio = alice_hf_sample['audio']
Audio(data=alice_audio['array'], rate=alice_audio['sampling_rate'])

/opt/conda/envs/proj/envs/waveglow/lib/python3.9/site-packages/librosa/util/decorators.py:88: UserWarning: Empty filters detected in mel frequency basis. Some channels will produce empty responses. Try increasing your sampling rate (and fmax) or reducing n_mels.
  return f(*args, **kwargs)


In [7]:
bob_hf_sample = asr.train_dataset.hf_ds[1]
bob_sample = asr.train_dataset[1]
bob_audio = bob_hf_sample['audio']
Audio(data=bob_audio['array'], rate=bob_audio['sampling_rate'])

In [4]:
with shelve.open(STYLE_TRANSFER_ARCHIVE) as db:
    results = db['374-180298-0000.flac:374-180298-0001.flac']

In [10]:
from types import SimpleNamespace
cfg = SimpleNamespace(**config)

In [29]:
import numpy as np

In [33]:
def transform_wav(self, wav, sr):
    wav = librosa.resample(wav, orig_sr=sr, target_sr=self.sr)

    mel_feats = librosa.feature.melspectrogram(y=wav, sr=self.sr, 
        n_mels=self.n_mels, n_fft=self.n_fft,
        win_length=self.win_length, hop_length=self.hop_length,
        fmin=self.fmin, fmax=self.fmax)

    log_mel_feats = librosa.power_to_db(mel_feats)
    input_norm = torch.as_tensor(np.max(np.abs(log_mel_feats), axis=0, keepdims=True).T, dtype=torch.float32)
    log_mel_feats = librosa.util.normalize(log_mel_feats)

    input_feature = log_mel_feats.T

#     input_feature, input_length = pad_wav(input_feature, self.wav_max_length)
    input_length = 0
    input_feature = torch.as_tensor(input_feature, dtype=torch.float32)

    return input_feature, input_norm, input_length

content_mels, norm, _ = transform_wav(cfg, bob_audio['array'], cfg.sr)

In [34]:
content_mels.shape, norm.shape

(torch.Size([1006, 80]), torch.Size([1006, 1]))

In [35]:
spectrogram = (content_mels.detach().cpu() * norm).T.numpy() # posttransfer_mels
power = librosa.db_to_power(spectrogram)
posttransfer_audio = librosa.feature.inverse.mel_to_audio(power, sr=cfg.sr, 
            n_fft=cfg.n_fft,
            win_length=cfg.win_length, hop_length=cfg.hop_length,
            fmin=cfg.fmin, fmax=cfg.fmax)

In [36]:
Audio(data=posttransfer_audio, rate=cfg.sr)